## Pydantic을 사용하여 prompt 작성하기

- `PydanticOutputParser`을 사용하여 긍/부정 분류를 한다.

- TEST

In [12]:
import pandas as pd
import json
from typing import List, Dict
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from dotenv import load_dotenv
import re

load_dotenv()
model = ChatOpenAI(model_name="gpt-3.5-turbo")


In [8]:
class Personality(BaseModel):
    category: str = Field(description="MBTI유형 이름")
    info: List[str] = Field(description="MBTI유형의 특징")
    
personality_query = "랜덤 MBTI 유형의 특징을 출력해주세요."
    
output_parser = PydanticOutputParser(pydantic_object=Personality)

In [9]:


prompt = PromptTemplate(
    template = "Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)
chain = prompt | model | output_parser
chain.invoke({"query": personality_query})

Personality(category='INFP', info=['Idealistic', 'Creative', 'Empathetic', 'Sensitive'])

---

In [10]:
class Personality(BaseModel):
    category: str = Field(description="MBTI유형 이름")
    info: str = Field(description="MBTI유형의 특징")
    
personality_query = "랜덤 MBTI 유형의 특징을 출력해주세요."
    
output_parser = PydanticOutputParser(pydantic_object=Personality)

In [11]:


prompt = PromptTemplate(
    template = "Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)
chain = prompt | model | output_parser
chain.invoke({"query": personality_query})

Personality(category='INTJ', info='INTJ 유형은 독립적이고 논리적이며 창의적인 편입니다. 계획을 세우고 목표를 이루는 데 능숙하며 분석적인 사고를 가지고 있습니다.')

---

In [17]:

# class Sentiment_classifier(BaseModel):
#     text: str = Field(description="리뷰 글")
#     classify: Dict[str, str] = Field(description= "음식의 긍정, 부정을 분류한다.")
    
# sentiment_query = "리뷰 글의 음식별 긍/부정을 분류하여 출력하세요."
# output_parser = PydanticOutputParser(pydantic_object=Sentiment_classifier)

# template = """\
# # INSTRUCTION
# - 당신은 SENTENCE에서 음식별 긍/부정을 분류하는 역할입니다.
# - 유저 쿼리에 대답하세요.\n{query}\n

# # SENTENCE: 양념게장비빔밥은 맛있지만 튀김은 맛없었습니다.
# """

# prompt = PromptTemplate.from_template(template)

# chain = prompt | model | output_parser
# response = chain.invoke({"query": sentiment_query})

---

- 연구원님의 깃허브 따라해보기


In [18]:
from typing import Dict, List, Optional

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

load_dotenv()
model = ChatOpenAI(model_name="gpt-3.5-turbo")

In [20]:
class ActionModel(BaseModel):
    positive: List[str] = Field(description="긍정을 나타내는 키워드")
    negative: List[str] = Field(description="부정을 나타내는 키워드")
    
class ReviewModel(BaseModel):
    category: str = Field(description="CATEGORY 문자열")
    action: ActionModel
    
parser = PydanticOutputParser(pydantic_object = ReviewModel)

template = """\
# INSTRUCTION
- 당신은 SENTENCE에서 CATEGORY에 따라 긍/부정에 해당하는 키워드를 분류하는 역할입니다.
- 키워드는 긍/부정의 대상이며, 명사만 추출하세요.
- FORMAT에 맞춰 답변하세요.

# FORMAT: {format_instructions}
# NOUNS: {nouns}
# CATEGORY: {category}
# SENTENCE: {sentence}
"""

nouns = "직원 친절 음식 맛 대한민국 호텔 뷔페 뭐 조금 입장 융통 부패 혁명"
category = "맛"
sentence = "스테이크는 오늘은 맛있어요 파크뷰는 대게 맛집 가격이 올라서 솔직히 155만 원 대비 가성비는 이 젠 별로 인 것 같아 요 발레할 경우에만 주차 편해요 비 오는 오늘 같은 경우 발레 안했음"
prompt = PromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions = parser.get_format_instructions())
output_parser = PydanticOutputParser(pydantic_object=ReviewModel)
chain = prompt | model | output_parser
chain.invoke({"nouns":nouns, "category": category, "sentence": sentence})

ReviewModel(category='맛', action=ActionModel(positive=['스테이크', '대게'], negative=['가격']))

In [21]:
class ActionModel(BaseModel):
    positive: List[str] = Field(description="긍정을 나타내는 키워드")
    negative: List[str] = Field(description="부정을 나타내는 키워드")
    
class ReviewModel(BaseModel):
    category: str = Field(description="CATEGORY 문자열")
    action: ActionModel
    
parser = PydanticOutputParser(pydantic_object = ReviewModel)

template = """\
# INSTRUCTION
- 당신은 SENTENCE에서 CATEGORY에 따라 긍/부정에 해당하는 키워드를 분류하는 역할입니다.
- 키워드는 긍/부정의 대상이며, 명사만 추출하세요.
- FORMAT에 맞춰 답변하세요.

# FORMAT: {format_instructions}
# CATEGORY: {category}
# SENTENCE: {sentence}
"""

category = "맛"
sentence = "스테이크는 오늘은 맛있어요 파크뷰는 대게 맛집 가격이 올라서 솔직히 155만 원 대비 가성비는 이 젠 별로 인 것 같아 요 발레할 경우에만 주차 편해요 비 오는 오늘 같은 경우 발레 안했음"
prompt = PromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions = parser.get_format_instructions())
output_parser = PydanticOutputParser(pydantic_object=ReviewModel)
chain = prompt | model | output_parser
chain.invoke({"category": category, "sentence": sentence})

ReviewModel(category='맛', action=ActionModel(positive=['스테이크', '대게'], negative=['가성비']))